In [1]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
from pulp import GLPK
import pandas as pd
import math 

In [8]:
#Load data
dataFrames = pd.read_csv("./PPL_sample/Week1/CO3005_003904_DH_HK201-Quiz Introduction (T2 2892020)-dixm.csv", sep=',')

In [9]:
dataFrames.describe()

,Tên,Họ,Tình trạng,Đã bắt đầu vào lúc,Đã hoàn thành,Thời gian thực hiện,"Điểm/10,00","Q. 1 /1,00","Q. 2 /1,00","Q. 3 /1,00","Q. 4 /1,00","Q. 5 /1,00","Q. 6 /1,00","Q. 7 /1,00","Q. 8 /1,00","Q. 9 /1,00","Q. 10 /1,00"
count,99,98,98,98,98,98,99,99,99,99,99,99,99,99,99,99,99
unique,68,91,1,10,8,79,40,4,4,3,5,4,5,4,5,4,4
top,Trung,Nguyễn Hoàng,Đã hoàn thành,28 September 2020 1:05 PM,28 September 2020 1:19 PM,14 phút 35 giây,"9,17","1,00","1,00","1,00","0,50","1,00","1,00","1,00","0,67","1,00","1,00"
freq,4,4,98,68,51,4,10,73,73,89,59,82,76,41,52,87,71


In [10]:
#Students information
students = dataFrames[:-1].loc[:,'Tên':'Họ']

In [14]:
#Drop the last line and select only question scores
scores = dataFrames[:-1].loc[:,'Q. 1 /1,00':'Q. 10 /1,00']

In [15]:
#Replace '-' with 0 and cast data to float type
scores = scores.replace('-','0').apply(lambda x: x.str.replace(',','.'))[:].astype(float)

In [16]:
#Rename the column
scores.columns = [str(i) for i in range(1,11)]

In [17]:
# Score data
scores

,1,2,3,4,5,6,7,8,9,10
0,1.00,1.00,1.00,1.0,1.00,1.0,0.5,0.33,1.0,1.00
1,1.00,-0.33,1.00,0.5,1.00,1.0,0.5,0.33,1.0,1.00
2,1.00,1.00,-0.33,0.5,1.00,0.5,0.0,0.67,1.0,1.00
3,1.00,1.00,1.00,1.0,1.00,1.0,1.0,0.67,1.0,1.00
4,-0.33,1.00,1.00,0.5,1.00,1.0,0.0,0.67,1.0,1.00
...,...,...,...,...,...,...,...,...,...,...
93,-0.33,1.00,1.00,0.5,1.00,1.0,0.0,0.67,1.0,-0.33
94,-0.33,-0.33,-0.33,0.0,-0.33,1.0,0.0,0.67,1.0,1.00
95,1.00,1.00,1.00,0.5,1.00,1.0,0.5,1.00,1.0,1.00
96,-0.33,-0.33,-0.33,0.5,-0.33,0.5,0.0,0.00,1.0,1.00


In [33]:
#student number
studentNumber = scores.count()['1']
#number of member in each group
k = 4
#number of group
groupNumber = math.ceil(studentNumber / k)
lastGroupNumber = studentNumber-k*(groupNumber-1)
questionList = [str(i) for i in range(1,11)]

25


##Declare variables

In [40]:
#x[i,j] whether student x is in group j
x = {}
for i in range(0, studentNumber):
    x[i] = {}
    for j in range(0, groupNumber):
        x[i][j] = LpVariable('x_'+str(i)+','+str(j),cat="Binary")
print(x[5])

{0: x_5,0, 1: x_5,1, 2: x_5,2, 3: x_5,3, 4: x_5,4, 5: x_5,5, 6: x_5,6, 7: x_5,7, 8: x_5,8, 9: x_5,9, 10: x_5,10, 11: x_5,11, 12: x_5,12, 13: x_5,13, 14: x_5,14, 15: x_5,15, 16: x_5,16, 17: x_5,17, 18: x_5,18, 19: x_5,19, 20: x_5,20, 21: x_5,21, 22: x_5,22, 23: x_5,23, 24: x_5,24}


In [42]:
#c[j,t] the quantity measuring the effect of grouping students into group j, toward exercise t
c = {}
for j in range(0, groupNumber):
    c[j]={}
    for t in questionList:
        c[j][t]=LpVariable('c_'+str(j)+','+t,lowBound=0)

{'1': c_0,1, '2': c_0,2, '3': c_0,3, '4': c_0,4, '5': c_0,5, '6': c_0,6, '7': c_0,7, '8': c_0,8, '9': c_0,9, '10': c_0,10}


In [21]:
prob = LpProblem("Mixed_Problem", LpMaximize)

In [22]:
for j in range(0, groupNumber):
    # Excercise related constraints:
    for t in questionList:
        #c[j,t]=min(10, sum(x[i,j]*score[i,t]))
        prob += c[j][t] <= 10 
        #c[j,t]=min(10, sum(x[i,j]*score[i,t]))
        prob += c[j][t] <= lpSum([x[i][j] * scores.loc[i][t] for i in range(0, studentNumber)])
    # Each group has exactly k students or lastGroupNumber if it is the last group
    if j < groupNumber-1:
        prob += lpSum([x[i][j] for i in range(0, studentNumber)]) == k 
    else:
        prob += lpSum([x[i][j] for i in range(0,studentNumber)]) == lastGroupNumber
        
for i in range(0,studentNumber):
    # Each student belongs to only one group
    prob += lpSum([x[i][j] for j in range(0, groupNumber)]) == 1


## Declare objective function

In [25]:
#The objective function is the sum of all c
prob += lpSum([cjt for cj in c.values() for cjt in cj.values()])

In [29]:
prob.solve(solver=GLPK(msg=False))

1

In [30]:
prob.objective.value()

680.9999999999994

In [31]:
solution={}
for i in range(0,studentNumber):
  for j in range(0,groupNumber):
    if not(j in solution):
      solution[j]=[]
    if x[i][j].value()==1:
      solution[j].append((i,students.loc[i]['Họ']+' '+students.loc[i]['Tên']))
      break
    
print(solution)

{0: [(8, 'Võ Hoàng Hải Nam'), (38, 'Trần Đình Vĩnh Thụy'), (79, 'Trần Khánh Tùng'), (80, 'Thái Duy Vũ')], 1: [(22, 'Lê Thanh Triều'), (41, 'Trương Việt Dũng'), (49, 'Nguyễn Thành Lưu'), (83, 'Nguyễn Văn Hoàn')], 2: [(24, 'Phạm Văn Thành'), (27, 'Trương Công Thành'), (74, 'Phạm Công Thiện'), (76, 'Ngô Lê Gia Thuấn')], 3: [(37, 'Phan Khánh Thịnh'), (72, 'Mai Văn Duyên'), (75, 'Lò Nhật Tân'), (95, 'Lê Bá Thông')], 4: [(11, 'Nguyễn Đỗ Quốc Duy'), (33, 'Sỳ Tùng An'), (68, 'Đặng Thành Ngân'), (93, 'Phan Quốc Long')], 5: [(16, 'Đỗ Lê Quang Trung'), (52, 'Lê Quang Duy'), (58, 'Nguyễn Phúc Nguyên'), (64, 'Phạm Quốc Trung')], 6: [(1, 'Lê Đức Huy'), (45, 'Hồ Thiên Long'), (77, 'Lăng Văn Dương'), (82, 'Trần Văn Viển')], 7: [(14, 'Lưu Văn Tiến'), (42, 'Võ Quang Nhật'), (54, 'Nguyễn Nhật Tân'), (63, 'Hồ Quang Khải')], 8: [(56, 'Nguyễn Duy Thìn'), (70, 'Lê Võ Hoàng Ân'), (71, 'Trần Nhất Tín'), (97, 'Huỳnh Tuấn Anh')], 9: [(19, 'Hoàng Vương'), (55, 'Võ Văn Toàn'), (67, 'Đinh Quang Trung'), (94, 'Dương